In [ ]:
# Imports and important constants
import tensorflow as tf
from keras import layers, metrics
from keras import losses
import numpy as np
import util
import time
from random import shuffle

# other classifiers for comparison
from sklearn import svm
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
import sklearn.metrics
from sklearn.naive_bayes import (
    BernoulliNB,
    ComplementNB,
    MultinomialNB,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

batch_size = 32
seed = 42
max_features = 1000
embedding_dim = 16
sequence_length = 250
epochs = 10


In [ ]:
# Import aclImbd
# This downloads a lot of text files, half of which are never referenced. 
# For more information, please see the function itself
util.prepare_data_dir()

In [2]:
# Define vectorization layer - embedding specfic
vectorize_layer = layers.TextVectorization(standardize=util.custom_standardization, max_tokens=max_features, output_mode='int', output_sequence_length=sequence_length)

def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label


In [ ]:
# Setup NLTK - scikit-learn specfic
# NLTK dataset for comparison, using same number of features. Very slow when compared to keras' text vectorization for the same number of tokens!
nltk.download(['vader_lexicon', 'movie_reviews','punkt', 'stopwords', 'names', 'averaged_perceptron_tagger'])

top_positive, top_negative = util.prepare_nltk(max_features)
nltk_features = [
    (util.nltk_extract_features(nltk.corpus.movie_reviews.raw(review), top_positive), "pos")
    for review in nltk.corpus.movie_reviews.fileids(categories=["pos"])
]
nltk_features.extend([
    (util.nltk_extract_features(nltk.corpus.movie_reviews.raw(review), top_positive), "neg")
    for review in nltk.corpus.movie_reviews.fileids(categories=["neg"])
])

In [ ]:
# Prepare datasets and adapt vectorization layer - embedding specfic
raw_train_ds = tf.keras.utils.text_dataset_from_directory('aclImdb/train', batch_size=batch_size, validation_split=0.2, subset='training', seed=seed)
raw_val_ds = tf.keras.utils.text_dataset_from_directory('aclImdb/train', batch_size=batch_size, validation_split=0.2, subset='validation', seed=seed)
raw_test_ds = tf.keras.utils.text_dataset_from_directory('aclImdb/test', batch_size=batch_size)
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [4]:
# Cache datasets for fast retrieval to speed up training - embedding specfic
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [5]:
# Embedding model
model_embedding = tf.keras.Sequential([
  layers.Embedding(max_features, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model_embedding.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0), metrics.AUC(from_logits=True)])

In [11]:
# NTLK models
classifiers = {
    "BernoulliNB": BernoulliNB(),
    "ComplementNB": ComplementNB(),
    "MultinomialNB": MultinomialNB(),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "LogisticRegression": LogisticRegression(),
    "MLPClassifier": MLPClassifier(max_iter=1000),
    "AdaBoostClassifier": AdaBoostClassifier(),
}

performance = {}

train_count = len(nltk_features) // 2
shuffle(nltk_features)
for name, sklearn_classifier in classifiers.items():
    classifier = nltk.classify.SklearnClassifier(sklearn_classifier)
    st = time.time()
    classifier.train(nltk_features[:train_count])
    ed = time.time()
    accuracy = nltk.classify.accuracy(classifier, nltk_features[train_count:])

    y_true, y_score = [], []

    for i, (feats, label_true) in enumerate(nltk_features[train_count:]):
        label_predicted = classifier.classify(feats)
        y_true.append(1 if label_true == 'pos' else 0)
        y_score.append(1 if label_predicted == 'pos' else 0)

    # Precision-Recall AUC
    precision, recall, _ = sklearn.metrics.precision_recall_curve(y_true, y_score, pos_label=1)
    pr_auc = sklearn.metrics.auc(recall, precision)
    fpr, tpr, _ = sklearn.metrics.roc_curve(y_true, y_score, pos_label=1)
    roc_auc = sklearn.metrics.auc(fpr, tpr)

    roc_auc = sklearn.metrics.auc(fpr, tpr)
    performance[name] = {}
    performance[name]['acc'] = accuracy
    performance[name]['time'] = ed-st
    performance[name]['auc'] = roc_auc

In [ ]:
# Evaluate Embedding Model
start = time.time()
history = model_embedding.fit(train_ds, validation_data=val_ds, epochs=epochs)
end = time.time()
util.evaluate_model_keras(model_embedding, test_ds, history)
print(f"Training Time Embedding: {end-start}")


In [ ]:
# Evaluate NLTK models
for name in performance:
    print(f'Model: {name}')
    acc = performance[name]['acc']
    time = performance[name]['time']
    auc = performance[name]['auc']
    print(f'   Accuracy: {acc}')
    print(f'   auROC: {auc}')
    print(f'   Train Time: {time}')